In [ ]:
%pip install databricks-vectorsearch

In [ ]:
from common import utils
from common import vector_search

from databricks.vector_search.client import VectorSearchClient

client = VectorSearchClient()
index = None
try:
    index = client.get_index(vector_search.ENDPOINT_NAME, vector_search.INDEX_NAME)
    utils.logger().info("vector search delta sync index exists: %s", vector_search.INDEX_NAME)
except Exception:
    utils.logger().info(
        "creating vector search delta sync index: %s", vector_search.INDEX_NAME
    )
    client.create_delta_sync_index_and_wait(
        #endpoint_name=ENDPOINT_NAME,
        index_name=vector_search.INDEX_NAME,
        source_table_name=vector_search.SOURCE_TABLE_NAME,
        pipeline_type="TRIGGERED",
        primary_key="search_id",
        embedding_source_column="text",
        embedding_model_endpoint_name="databricks-gte-large-en",
        columns_to_sync=["id", "text", "path", "content_hash"],
    )
if index:
    # TODO: Add conditional sync logic
    utils.logger().info("syncing vector search delta sync index:%s", index.describe())
    index.sync()